In [105]:
import numpy as np
import pandas as pd
import inspect
import os
np.random.seed(0)

#Activation functions

def sin_function(x):
    return np.sin(x)

def sin_function_derivative(x):
    return -np.sin(x)

def relu(x):
    return np.maximum(0, x)

def relu_derivative(x):
    return np.where(x <= 0, 0, 1)

def tanh(x):
    return np.tanh(x)

def tanh_derivative(x):
    return -2 * np.tanh(x) * (1 - np.tanh(x)**2)

#Function to approximate
def f(x):
    return np.sin(3*np.pi*x + 3*np.pi/20) * np.cos(2*np.pi*x + np.pi/10) + 2

In [106]:
#Function to build the mass matrix - M
from scipy.integrate import quad

def mass_matrix(func,sigma, weights, biases, n):
    M = np.zeros([n, n])
    
    for i in range(0, n):
        for j in range(0, n):
            M[i, j] = quad(func, 0, 1, args=(weights, biases,i+1,j+1,sigma), limit=1000)[0]

    return M

In [107]:
#Single basis function - Neural Network
def single_neural_net(x,weights,biases,i, sigma): 

    return sigma(weights[i]*x+biases[i])


In [108]:
#Basis function multiplication helper for quadrature
def double_neural_net(x,weights,biases,i,j, sigma): 

    return single_neural_net(x,weights,biases,i,sigma)*single_neural_net(x,weights,biases,j,sigma)

In [109]:
#Bias helper function for quadrature
def b_neural_net(x, weights, biases,i, sigma,f):
    return single_neural_net(x,weights,biases,i, sigma)*f(x)


In [110]:
#quadrature for the bias matrix - B
def b_matrix(func,sigma, weights, biases,f, neurons):
    b = np.zeros([neurons, 1])
    for i in range(0, neurons):
            b[i] = quad(func, 0, 1, args=(weights, biases,i,sigma,f), limit=1000)[0]

    return b

In [111]:
#Function to build the approximation vector
def u_nn(x, weights, biases, sigma,collocation, neurons, a):
    u = np.zeros(collocation)
    
    for j in range(0,collocation):
        for i in range(0,neurons):
            
            u[j] += single_neural_net(x[j], weights, biases, i, sigma)*a[i]
        
    return u


In [112]:
#helper function for eigenvalues and eigenvectors
from scipy.linalg import eig
def get_eig(M):
    D,V = eig(M, left=False, right=True)


    idx = D.argsort()[::-1]   
    D = D[idx]
    V = V[:,idx]

    D = np.flip(D)
    V = np.fliplr(V)

    return D,V

In [113]:
#Function to calculate L2 loss
def l2_loss(f_x, u_nn_result):
    return np.linalg.norm(f_x - u_nn_result)**2

In [114]:
#Function to build the Phi matrix, made up of n basis functions
def phi_matrix(x,weights,biases,sigma,neurons,collocation):
    phi = np.zeros((collocation,neurons))
    for i in range(0,neurons):
        for j in range(0,collocation):

            phi[j,i] = sigma(weights[i]*x[j]+biases[i])
    return phi


In [115]:
def evaluate_func_approx(x,f, weights, biases, sigma, neurons, collocation,verbose=False):
    phi = phi_matrix(x,weights,biases,sigma,neurons,collocation)
    f_func = f(x)
    a = np.linalg.pinv(phi)@f_func

    u = u_nn(x, weights, biases, sigma, collocation,neurons, a)

    loss = l2_loss(f_func, u)
    if verbose:
        print("="*20)
        print("Sigma:", sigma.__name__)
        print("Number of collocation points:", collocation)
        print("Number of neurons:", neurons)
        print("L2 Loss:", loss)
        print("="*20)
    return loss

In [116]:
def export_list(data, list_name, sheet_name, directory=None, columns=None, overwrite=False):
    default_directory = '/mnt/c/Git_Repos/Function_approximation/Results'
    file_name = f"{list_name}.xlsx"
    
    if columns is None:
        columns = ["Loss", "# of Collocation", "# of Neurons", "Activation Function", "R - Sample Range"]
    
    if directory is None:
        directory = default_directory
    
    full_path = os.path.join(directory, file_name)
    df = pd.DataFrame(data, columns=columns)

    if os.path.exists(full_path):
        try:
            # If the file exists
            with pd.ExcelWriter(full_path, engine='openpyxl', mode='a') as writer:
                if overwrite:
                    # If overwrite is True, delete the sheet if it exists
                    if sheet_name in writer.book.sheetnames:
                        writer.book.remove(writer.book[sheet_name])
                # Write the DataFrame to the Excel file
                df.to_excel(writer, sheet_name=sheet_name, index=False)
        except ValueError as e:
            # If the sheet already exists and overwrite is False, print a message and continue
            print(f"Sheet name '{sheet_name}' already exists in the file '{file_name}'")
    else:
        # If the file does not exist, create a new one
        df.to_excel(full_path, index=False, sheet_name=sheet_name)

    print(f"Data exported to {full_path}")

In [117]:
def compute_loss_values(neurons, collocations, sigmas, Rs, sheet_name, xmin=0, xmax=1, overwrite=False, verbose=False):
    loss_values = []
    #np.random.seed(0)  # Set a seed for reproducibility
    for sigma in sigmas:
        for neuron in neurons:
            for collocation in collocations:
                for R in Rs:
                    x = np.linspace(xmin, xmax, collocation)
                    weights = np.random.uniform(-R, R, neuron)
                    biases = np.random.uniform(-R, R, neuron)
                    loss = evaluate_func_approx(x, f, weights, biases, sigma, neuron, collocation, verbose=verbose)
                    loss_values.append((loss, collocation, neuron, sigma.__name__,R))  # Add activation function name
    export_list(loss_values, "Finite_Neuron_Method_Results", sheet_name, overwrite=overwrite)
    return loss_values

In [118]:
def print_loss_by_collocation(loss_values):
    """
    Prints the loss values grouped by collocation point, activation function, and range R.
    
    Parameters:
    loss_values (list of tuples): Each tuple contains (loss, collocation, neuron, activation_func_name, R).
    """
    # Initialize the dictionary to store data for each collocation point
    collocation_loss_data = {collocation: [] for _, collocation, _, _, _ in loss_values}

    # Populate the dictionary with (neuron, loss, activation_func_name, R) tuples for each collocation point
    for loss, collocation, neuron, activation_func_name, R in loss_values:
        collocation_loss_data[collocation].append((neuron, loss, activation_func_name, R))

    # Iterate over the unique activation function names
    for activation_func_name in set(activation_func_name for _, _, _, activation_func_name, _ in loss_values):
        print(f"Activation Function: {activation_func_name}")
        
        # Iterate over the collocation points and print the relevant data
        for collocation, data in collocation_loss_data.items():
            print(f"# of Collocation points: {collocation}")
            
            # Print the neurons, loss values, and R for the current activation function
            for neuron, loss, act_func_name, R in data:
                if act_func_name == activation_func_name:
                    print(f"  # of Neurons: {neuron}, Loss: {loss:.10e}, R: {R}")



In [119]:
def print_loss_by_neuron(loss_values):
    """
    Prints the loss values grouped by neuron, activation function, and range R.
    
    Parameters:
    loss_values (list of tuples): Each tuple contains (loss, collocation, neuron, activation_func_name, R).
    """
    # Initialize the dictionary to store data for each neuron
    neuron_loss_data = {neuron: [] for _, _, neuron, _, _ in loss_values}

    # Populate the dictionary with (collocation, loss, activation_func_name, R) tuples for each neuron
    for loss, collocation, neuron, activation_func_name, R in loss_values:
        neuron_loss_data[neuron].append((collocation, loss, activation_func_name, R))

    # Iterate over the unique activation function names
    for activation_func_name in set(activation_func_name for _, _, _, activation_func_name, _ in loss_values):
        print(f"Activation Function: {activation_func_name}")
        
        # Iterate over the neurons and print the relevant data
        for neuron, data in neuron_loss_data.items():
            print(f"# of Neurons: {neuron}")
            
            # Print the collocation points, loss values, and R for the current activation function
            for collocation, loss, act_func_name, R in data:
                if act_func_name == activation_func_name:
                    print(f"  # of Collocation points: {collocation}, Loss: {loss:.10e}, R: {R}")




In [120]:
def print_loss_by_R(loss_values):
    """
    Prints the loss values grouped by the range R, activation function, collocation points, and neurons.
    
    Parameters:
    loss_values (list of tuples): Each tuple contains (loss, collocation, neuron, activation_func_name, R).
    """
    # Initialize the dictionary to store data for each R value
    R_loss_data = {R: [] for _, _, _, _, R in loss_values}

    # Populate the dictionary with (collocation, neuron, loss, activation_func_name) tuples for each R value
    for loss, collocation, neuron, activation_func_name, R in loss_values:
        R_loss_data[R].append((collocation, neuron, loss, activation_func_name))

    # Iterate over the unique activation function names
    for activation_func_name in set(activation_func_name for _, _, _, activation_func_name, _ in loss_values):
        print(f"Activation Function: {activation_func_name}")
        
        # Iterate over the R values and print the relevant data
        for R, data in R_loss_data.items():
            print(f"Range R: {R}")
            
            # Print the collocation points, neurons, and loss values for the current activation function
            for collocation, neuron, loss, act_func_name in data:
                if act_func_name == activation_func_name:
                    print(f"  # of Collocation points: {collocation}, # of Neurons: {neuron}, Loss: {loss:.10e}")

In [121]:
collocation = 50
neurons = 25
sigma = sin_function2_derivative
x = np.linspace(0,1,collocation)
weights = np.random.uniform(-5,5,neurons)
biases = np.random.uniform(-5,5,neurons)
test = evaluate_func_approx(x,f, weights, biases, sigma, neurons, collocation,verbose=True)

Sigma: sin_function2_derivative
Number of collocation points: 50
Number of neurons: 25
L2 Loss: 1.8921620734168687e-05


# Test 1 - Increasing #of neurons
Keeping collocations fixed at 50, and using ReLU, tanh, and sin as activation functions

In [122]:
neurons = [5,10,20,30,40,50,100]
sigmas = [relu_derivative,tanh_derivative,sin_function_derivative]
collocations = [50]
Rs = [1]
loss_values_increasing_neurons = compute_loss_values(neurons, collocations, sigmas, Rs, "Increasing # of Neurons", overwrite=False, verbose=False)
print_loss_by_R(loss_values_increasing_neurons)
#loss_values

Sheet name 'Increasing # of Neurons' already exists in the file 'Finite_Neuron_Method_Results.xlsx'
Data exported to /mnt/c/Git_Repos/Function_approximation/Results/Finite_Neuron_Method_Results.xlsx
Activation Function: relu_derivative
Range R: 1
  # of Collocation points: 50, # of Neurons: 5, Loss: 5.7329351727e+00
  # of Collocation points: 50, # of Neurons: 10, Loss: 4.2788469935e+00
  # of Collocation points: 50, # of Neurons: 20, Loss: 1.7828646175e+00
  # of Collocation points: 50, # of Neurons: 30, Loss: 2.1174095914e+00
  # of Collocation points: 50, # of Neurons: 40, Loss: 2.5218548293e+00
  # of Collocation points: 50, # of Neurons: 50, Loss: 8.0051356822e-01
  # of Collocation points: 50, # of Neurons: 100, Loss: 4.0705972680e-01
Activation Function: sin_function_derivative
Range R: 1
  # of Collocation points: 50, # of Neurons: 5, Loss: 5.3105737543e+00
  # of Collocation points: 50, # of Neurons: 10, Loss: 2.3139909258e-01
  # of Collocation points: 50, # of Neurons: 20, L

/home/zephunix/miniconda3/envs/tf/lib/python3.10/site-packages/openpyxl/workbook/child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")


# Test 2 - Increasing Collocation points
Keeping neurons fixed at 20, and the same activation functions

In [123]:
neurons = [20]
sigmas = [relu_derivative,tanh_derivative,sin_function_derivative]
collocations = [10,50,100,200,500]
Rs = [1]
loss_values = compute_loss_values(neurons, collocations,sigmas,Rs,"Increasing # of Collocation Points",overwrite=False,verbose=False)
print_loss_by_neuron(loss_values)

Sheet name 'Increasing # of Collocation Points' already exists in the file 'Finite_Neuron_Method_Results.xlsx'
Data exported to /mnt/c/Git_Repos/Function_approximation/Results/Finite_Neuron_Method_Results.xlsx
Activation Function: relu_derivative
# of Neurons: 20
  # of Collocation points: 10, Loss: 1.2553876083e+00, R: 1
  # of Collocation points: 50, Loss: 4.5440930105e+00, R: 1
  # of Collocation points: 100, Loss: 1.1309512322e+01, R: 1
  # of Collocation points: 200, Loss: 2.4029897539e+01, R: 1
  # of Collocation points: 500, Loss: 2.9035023417e+01, R: 1
Activation Function: sin_function_derivative
# of Neurons: 20
  # of Collocation points: 10, Loss: 1.1996700803e-04, R: 1
  # of Collocation points: 50, Loss: 4.4664400298e-02, R: 1
  # of Collocation points: 100, Loss: 9.3847188837e-02, R: 1
  # of Collocation points: 200, Loss: 1.6576035139e-01, R: 1
  # of Collocation points: 500, Loss: 3.8670733803e-01, R: 1
Activation Function: tanh_derivative
# of Neurons: 20
  # of Colloca

/home/zephunix/miniconda3/envs/tf/lib/python3.10/site-packages/openpyxl/workbook/child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")


In [124]:
import pandas as pd

# Convert list to DataFrame
df = pd.DataFrame(loss_values, columns=['Loss', 'Collocation', 'Neurons', 'Activation Function', 'R'])

# Sort DataFrame by 'Loss' column
sorted_df = df.sort_values(by='Loss')

# print("Sorted by Loss")
# print(sorted_df)
# print("="*80)
# print("")
# print("Filtered for Sin Activation Function")
# filtered_df = df[df["Activation Function"] == "sin_function"]

# print(filtered_df)

In [125]:
#print_loss_by_neuron(loss_values)

# Test 3 - Increasing the sampling range R
Keeping neurons and collocations fixed, increasing R to increase chance that w_i and b_i are further away from w_j and b_j

In [126]:
neurons = [30]
sigmas = [tanh,relu,sin_function]
sigmas_derivative = [tanh_derivative,relu_derivative,sin_function_derivative]
collocations = [100]
Rs = [1,2,4,5,10]
loss_values = compute_loss_values(neurons, collocations,sigmas,Rs,"Increasing R",overwrite=False,verbose=False)
derivative_loss_values = compute_loss_values(neurons, collocations,sigmas_derivative,Rs,"Increasing R",overwrite=True,verbose=False)

/home/zephunix/miniconda3/envs/tf/lib/python3.10/site-packages/openpyxl/workbook/child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")


Sheet name 'Increasing R' already exists in the file 'Finite_Neuron_Method_Results.xlsx'
Data exported to /mnt/c/Git_Repos/Function_approximation/Results/Finite_Neuron_Method_Results.xlsx
Data exported to /mnt/c/Git_Repos/Function_approximation/Results/Finite_Neuron_Method_Results.xlsx


/home/zephunix/miniconda3/envs/tf/lib/python3.10/site-packages/openpyxl/workbook/child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")


In [127]:
derivative_loss_values


[(0.0003455134170967644, 100, 30, 'tanh_derivative', 1),
 (5.432211212234588e-05, 100, 30, 'tanh_derivative', 2),
 (3.5348750496849822e-06, 100, 30, 'tanh_derivative', 4),
 (3.811423845927587e-05, 100, 30, 'tanh_derivative', 5),
 (6.170415123650935e-10, 100, 30, 'tanh_derivative', 10),
 (5.7168269891407615, 100, 30, 'relu_derivative', 1),
 (5.609330132244446, 100, 30, 'relu_derivative', 2),
 (6.575301635364382, 100, 30, 'relu_derivative', 4),
 (5.416792150516557, 100, 30, 'relu_derivative', 5),
 (6.335727090258714, 100, 30, 'relu_derivative', 10),
 (0.08422150807032926, 100, 30, 'sin_function_derivative', 1),
 (0.0017428802773873093, 100, 30, 'sin_function_derivative', 2),
 (1.9056814726028912e-05, 100, 30, 'sin_function_derivative', 4),
 (2.113425450170623e-05, 100, 30, 'sin_function_derivative', 5),
 (7.846881351737049e-05, 100, 30, 'sin_function_derivative', 10)]

In [128]:
loss_values

[(3.309896953251292e-05, 100, 30, 'tanh', 1),
 (0.00048619058656185893, 100, 30, 'tanh', 2),
 (2.1674058882000564e-07, 100, 30, 'tanh', 4),
 (1.7280728325284913e-05, 100, 30, 'tanh', 5),
 (0.00014075296618092363, 100, 30, 'tanh', 10),
 (3.307905316179587, 100, 30, 'relu', 1),
 (6.814174842179699, 100, 30, 'relu', 2),
 (2.0275428797723904, 100, 30, 'relu', 4),
 (9.25612921059808, 100, 30, 'relu', 5),
 (8.911102213415985, 100, 30, 'relu', 10),
 (0.08528242352099909, 100, 30, 'sin_function', 1),
 (0.0017284368406900539, 100, 30, 'sin_function', 2),
 (1.602402138023601e-05, 100, 30, 'sin_function', 4),
 (3.992694431729737e-05, 100, 30, 'sin_function', 5),
 (0.0036673028336331143, 100, 30, 'sin_function', 10)]

In [129]:
comparison_values = loss_values + derivative_loss_values
df = pd.DataFrame(comparison_values, columns=["Loss", "# of Collocation", "# of Neurons", "Activation Function", "R - Sample Range"])

# Export the DataFrame to an Excel file
df.to_excel("combined_loss_values.xlsx", index=False)

print("Data exported to combined_loss_values.xlsx")

Data exported to combined_loss_values.xlsx


In [130]:
#print_loss_by_collocation(derivative_loss_values)

In [131]:
def compare_loss_values(loss_values, derivative_loss_values):
    function_dict = {(func, r): err for err, _, _, func, r in loss_values}
    derivative_dict = {(func, r): err for err, _, _, func, r in derivative_loss_values}

    def base_function_name(derivative_name):
        if derivative_name.endswith('_derivative'):
            return derivative_name.replace('_derivative', '')
        return None

    comparison_results = []
    for (func_name, r), err in derivative_dict.items():
        base_name = base_function_name(func_name)
        if base_name:
            base_err = function_dict.get((base_name, r))
            if base_err is not None:
                comparison_results.append((base_name, r, base_err, err))

    print(f"{'Function':<20}{'R Value':<10}{'Function Error':<20}{'Derivative Error':<25}")
    print("="*80)
    for base_name, r, base_err, der_err in comparison_results:
        print(f"{base_name:<20}{r:<10}{base_err:<20.10e}{der_err:<25.10e}")


In [132]:
compare_loss_values(loss_values,derivative_loss_values)

Function            R Value   Function Error      Derivative Error         
tanh                1         3.3098969533e-05    3.4551341710e-04         
tanh                2         4.8619058656e-04    5.4322112122e-05         
tanh                4         2.1674058882e-07    3.5348750497e-06         
tanh                5         1.7280728325e-05    3.8114238459e-05         
tanh                10        1.4075296618e-04    6.1704151237e-10         
relu                1         3.3079053162e+00    5.7168269891e+00         
relu                2         6.8141748422e+00    5.6093301322e+00         
relu                4         2.0275428798e+00    6.5753016354e+00         
relu                5         9.2561292106e+00    5.4167921505e+00         
relu                10        8.9111022134e+00    6.3357270903e+00         
sin_function        1         8.5282423521e-02    8.4221508070e-02         
sin_function        2         1.7284368407e-03    1.7428802774e-03         
sin_function

In [133]:
neurons = [10,20,30,50,100]
sigmas = [tanh,relu,sin_function]
collocations = [5,10,30,50,100]
Rs = [1,2,4,5,10]
loss_values = compute_loss_values(neurons, collocations,sigmas,Rs,"Full_tests",overwrite=True,verbose=False)

Data exported to /mnt/c/Git_Repos/Function_approximation/Results/Finite_Neuron_Method_Results.xlsx


/home/zephunix/miniconda3/envs/tf/lib/python3.10/site-packages/openpyxl/workbook/child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")


In [134]:
#print_loss_by_collocation(loss_values)

# Building Mass matrix

In [135]:
from scipy.integrate import quad

def mass_matrix(func,sigma, weights, biases, neurons):
    M = np.zeros([neurons, neurons])
    
    for i in range(0, neurons):
        for j in range(0, neurons):
            M[i, j] = quad(func, 0, 1, args=(weights, biases,i,j,sigma), limit=1000)[0]

    return M

In [148]:
neurons = 100
R = 10
collocation = 1000
x = np.linspace(0,1,collocation)
weights = np.random.uniform(-R,R,neurons)
biases = np.random.uniform(-R,R,neurons)
sigma = tanh_derivative


In [152]:
def create_linear_system(x, neurons, collocation, weights, biases, sigma, f):

    M = mass_matrix(double_neural_net,sigma, weights, biases, neurons)
    b = b_matrix(b_neural_net,sigma,weights, biases,f,neurons)
    a = np.linalg.solve(M,b)
    u = u_nn(x,weights,biases,sigma,collocation,neurons,a)
    
    loss = l2_loss(f(x), u)
    print("L2 Loss:", loss)

    return M, b, a, u


In [153]:
M, b, a, u = create_linear_system(x, neurons, collocation, weights, biases, sigma, f)

/tmp/ipykernel_23439/1981966750.py:8: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  u[j] += single_neural_net(x[j], weights, biases, i, sigma)*a[i]


L2 Loss: 3.0405575746047427e-05
